In [ ]:
##PROCESSING OF ITS SEQUENCES##

In [ ]:
#A list of files to assembly is generated.
ls *.fastq.gz | perl -pe 's/_R.*.fastq.gz//g' | sort | uniq > lista

In [ ]:
#Use assembly sh script
The "assembly.sh" scrip generates the works to assembly files from list.

In [ ]:
#!/bin/bash
# Use: bash assemblyCASPER.sh NOMBRE_TRABAJO

SEQS=$(pwd)
SALIDAS=$(pwd)
BIN=/usr/local/bin
COUNT=0

for FAA in `cat lista`
do
let COUNT=COUNT+1
echo "#!/bin/bash" >$*.$COUNT.scr
echo "#$ -cwd" >>$*.$COUNT.scr
echo "#$ -j y" >>$*.$COUNT.scr
echo "#$ -S /bin/bash" >>$*.$COUNT.scr

echo "$BIN/casper <(cat $SEQS/$FAA"_R1.fastq") <(cat($SEQS/$FAA"_R2.fastq") -o $FAA.assembly.fastq -o $FAA"_assembly"" >>$*.$COUNT.scr

done

In [ ]:
#Quality filter, remove sequence with Quality score lower than 20. 
#This step is made to remove low quality assemblies

#!/bin/bash
SEQS=$(pwd)
SALIDAS=$(pwd)
for FAA in `cat lista`
do
let COUNT=COUNT+1
echo "#!/bin/bash" >$*.$COUNT.scr
echo "#$ -cwd" >>$*.$COUNT.scr
echo "#$ -j y" >>$*.$COUNT.scr
echo "#$ -S /bin/bash" >>$*.$COUNT.scr

echo fastq_quality_filter -q 20 -p 80 -i "$SEQS/$FAA".fastq -o "$SEQS/$FAA"_tr.fastq >>$*.$COUNT.scr

done

In [ ]:
#Transform filtered sequences to fasta
for x in `cat lista`; do sed -n '1~4s/^@/>/p;2~4p' "$x"_tr.fastq > "$x"_ensamble.fasta

In [ ]:
Sequences names are changed with “header.fasta.numbers.pl”.

# Luis David Alcaraz 2013-04-11

my $prefix = $ARGV[0]; chomp $prefix;
my $f =  1;

my $fasta_file = $ARGV [1]; chomp $fasta_file;

my $fh;
open($fh, $fasta_file) or die "can't open $fasta_file: $!\n";
open(OUT, ">$fasta_file.numbered.fas") || die "can't open $fasta_file.numbered.f
as\n";

my %sequence_data;
while (read_fasta_sequence($fh, \%sequence_data)) {
   print OUT ">$sequence_data{header}\n$sequence_data{seq}\n";
}

close $fh;
close OUT;

sub read_fasta_sequence {
   my ($fh, $seq_info) = @_;

$seq_info->{seq} = undef; # clear out previous sequence

   # put the header into place
   $seq_info->{header} = $seq_info->{next_header} if $seq_info->{next_header};

   my $file_not_empty = 0;
   while (<$fh>) {
      $file_not_empty = 1;
      next if /^\s*$/;  # skip blank lines
      chomp;    

      if (/^>/) { # fasta header line
         my $h = $_;    
         $h =~ s/>/$prefix\_$f\ /;
     $f++;
         if ($seq_info->{header}) {
            $seq_info->{next_header} = $h;
            return $seq_info;
       
         }              
         else { # first time through only
            $seq_info->{header} = $h;
         }              
      }         
      else {    
         s/\s+//;  # remove any white space
         s/\n\n/\n/;
         $seq_info->{seq} .= $_;
      }         
   }    

   if ($file_not_empty) {
      return $seq_info;
   }    
   else {
      # clean everything up
      $seq_info->{header} = $seq_info->{seq} = $seq_info->{next_header} = undef;

      return;   
   }    
}

In [ ]:
#Run the script for each sample
for N in `ls *_ensamble.fasta`; do perl header.fasta.numbers.pl $N $N"_ensamble.fasta"

In [ ]:
#Concatenate all samples
cat *.numbered.fas > mezcal_fun.fas

In [ ]:
#Edit sequence name to leave only the first part, which refers to sample name. 
perl -i.bak -pe "s/\ .*//g" mezcal_fun.fas

In [ ]:
#Sequence clustering of OTUs at 97%  of identity was done with cd-hit-est, -c indicates the identity for clustering.
#!/bin/bash
#$ -cwd
#$ -j y
#$ -S /bin/bash
cd-hit-est -i mezcal_fun.fas -c 0.97 -o mezcal_fun.fasout -T 20 -M 0

In [ ]:
#The clustering file is converted in a file that can be read by QIIME.
perl -pne 's/\t//g;s/^.*,//g;s/\.\.\..*$//g;s/\n/\t/g;s/\>Cluster\ /\n/g;s/\>//g; eof && do{chomp; print "$_ \n"; exit}' mezcal_fun.fasout.clstr > mezcal_fun.otu
sed -i '1d' mezcal_fun.otu

In [ ]:
## Extract representative OTUs, -i indicates input file, -f is the fasta file with the sequences to be extracted. 

#!/bin/bash
#$ -cwd
#$ -j y
#$ -S /bin/bash
pick_rep_set.py -i mezcal_fun.otu -f mezcal_fun.fas -o mezcal_fun.rep.fna

In [ ]:
#Assign taxonomy against most recent UNITE database. We use the complete database to accuretly assign non fungal taxa

parallel_assign_taxonomy_blast.py -i mezcal_fun.rep.fna -o taxonomy -r /home/cristobal/DB/unite_8.3/sh_qiime_release_all_10.05.2021/sh_refs_qiime_ver8_97_all_10.05.2021.fasta -t /home/cristobal/DB/unite_8.3/sh_qiime_release_all_10.05.2021/sh_taxonomy_qiime_ver8_97_all_10.05.2021.txt

#Create a list of fungal ITS sequences
cat taxonomy/mezcal_fun.rep_tax_assignments.txt | grep "k__Fungi" | cut -f1 >ids_screened.txt

#Create a list of sequences that are not fungi
cat taxonomy/mezcal_fun.rep_tax_assignments.txt | grep -v "k__Fungi" | cut -f1 >ids_REMOVE_biom.txt

#Extract ITS sequences from ITS list and make a new representative file
perl -ne 'if(/^>(\S+)/){$c=$i{$1}}$c?print:chomp;$i{$_}=1 if @ARGV' ids_screened.txt mezcal_fun.rep.fna > mezcal_fun.screened.fna

In [ ]:
#Make biom object

make_otu_table.py -i mezcal_fun.otu -t taxonomy/mezcal_fun.rep_tax_assignments.txt -o mezcal_fun.biom 

#Remove singletons and non fungal sequences. Chimeras are removed later

filter_otus_from_otu_table.py -i mezcal_fun.biom -e ids_REMOVE_biom.txt -o mezcal_fun_screened.biom -n2 

In [ ]:
#Remove chimeras from representative sequences

#Identify chimera

parallel_identify_chimeric_seqs.py -m blast_fragments -i mezcal_fun.screened.fna -o mezcal_fun.chimera.txt -X mezcal_funblast --id_to_taxonomy_fp /home/cristobal/DB/unite_8.3/sh_qiime_release_all_10.05.2021/sh_taxonomy_qiime_ver8_97_all_10.05.2021.txt -r /home/cristobal/DB/unite_8.3/sh_qiime_release_all_10.05.2021/sh_refs_qiime_ver8_97_all_10.05.2021.fasta


#Filter chimeric sequences from biom table

filter_otus_from_otu_table.py -i mezcal_fun_screened.biom -e mezcal_fun.chimera.txt -o mezcal_fun_chimera.biom

In [ ]:
#Create tables
biom convert --to-tsv -i mezcal_fun_chimera.biom -o mezcal_fun.biom.tsv --table-type "Taxon table" --header-key=taxonomy

#Remove OTUs labeled as "None"
grep -v 'None' mezcal_fun.biom.tsv > mezcal_fun_filt.biom.tsv; mv mezcal_fun_filt.biom.tsv mezcal_fun.biom.tsv

In [ ]:
#Split taxonomy table from OTU table.
perl -pe 's/\; /\;/g' mezcal_fun.biom.tsv | awk '{print $1,"\t",$NF}' | perl -pe 's/\;/\t/g' > fun_mp_tax.tsv

#Split OTU abundance for sample from OTU table: 
cut -f 1-87 mezcal_fun.biom.tsv > fun_mp_otu.tsv